![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Train POS Tagger in French by Spark NLP
### Based on Universal Dependency `UD_French-GSD`


In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.3.4

In [ ]:
import sys
import time

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher


### Let's create a Spark Session for our app

In [ ]:
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Let's prepare our training datasets containing `token_posTag` like `de_DET`. You can download this data set from Amazon S3:

```
wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/fr/pos/UD_French/UD_French-GSD_2.3.txt -P /tmp
```

In [ ]:
! wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/fr/pos/UD_French/UD_French-GSD_2.3.txt -P /tmp

In [ ]:
from sparknlp.training import POS
training_data = POS().readDataset(spark, '/tmp/UD_French-GSD_2.3.txt', '_', 'tags')

In [ ]:
training_data.show()

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")\
    .setExceptions(["jusqu'", "aujourd'hui", "États-Unis", "lui-même", "celui-ci", "c'est-à-dire", "celle-ci", "au-dessus", "etc.", "sud-est", "Royaume-Uni", "ceux-ci", "au-delà", "elle-même", "peut-être", "sud-ouest", "nord-ouest", "nord-est", "Etats-Unis", "Grande-Bretagne", "Pays-Bas", "eux-mêmes", "porte-parole", "Notre-Dame", "puisqu'", "week-end", "quelqu'un", "celles-ci", "chef-lieu"])\
    .setPrefixPattern("\\A([^\\s\\p{L}\\d\\$\\.#]*)")\
    .setSuffixPattern("([^\\s\\p{L}\\d]?)([^\\s\\p{L}\\d]*)\\z")\
    .setInfixPatterns([
      "([\\p{L}\\w]+'{1})",
      "([\\$#]?\\d+(?:[^\\s\\d]{1}\\d+)*)",
      "((?:\\p{L}\\.)+)",
      "((?:\\p{L}+[^\\s\\p{L}]{1})+\\p{L}+)",
      "([\\p{L}\\w]+)"
    ])

posTagger = PerceptronApproach() \
    .setNIterations(1) \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("pos") \
    .setPosCol("tags")
    
pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector, 
    tokenizer,
    posTagger
])

In [ ]:
# Let's train our Pipeline by using our training dataset
model = pipeline.fit(training_data)

This is our testing DataFrame where we get some sentences in French. We are going to use our trained Pipeline to transform these sentence and predict each token's `Part Of Speech`.

In [ ]:
dfTest = spark.createDataFrame([
    "Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.",
    "On pourra toujours parler à propos d'Averroès de décentrement du Sujet."
], StringType()).toDF("text")

In [ ]:
predict = model.transform(dfTest)

In [ ]:
predict.select("token.result", "pos.result").show()